In [1]:
import requests
from bs4 import BeautifulSoup
from functools import reduce
import csv
import pandas as pd
import sqlite3

In [15]:
domain='https://books.toscrape.com/'
respose = requests.get(domain).text
soup = BeautifulSoup(respose,'html.parser')
rates_dic={'Zero':0,'One':1,'Two':2,'Three':3,'Four':4,'Five':5}
category_link = soup.find_all('ul',attrs={'class':'nav nav-list'})
     
for x in category_link:
    y = x.find_all('a')[1:]
    for z in y:
        c = z.get('href')
        a = domain+c
        category_name = z.get_text().strip()
        soup_cat=BeautifulSoup(requests.get(a).text,'html.parser')

        for article in soup_cat.find_all('article',attrs={'class':'product_pod'}):
            Book_name=article.find('h3').find('a').get('title')
            Price=float(article.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
            Rate=rates_dic[article.find('p').get('class')[1]]
            
            connection=sqlite3.connect('books.db')
            Cursor=connection.cursor()
            Cursor.execute('insert into books  (Book_name,Price,Rate,category_name)values (?,?,?,?)',[Book_name,Price,Rate,category_name])
            connection.commit()
            
        while soup_cat.find('li',attrs={'class':'next'}):
            new_page_url=reduce(lambda a,b:a+'/'+b,a.split('/')[:-1])+'/'+soup_cat.find('li',attrs={'class':'next'}).find('a').get('href')
            soup_cat=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
            for article in soup_cat.find_all('article',attrs={'class':'product_pod'}):
                Book_name=article.find('h3').find('a').get('title')
                Price=float(article.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
                Rate=rates_dic[article.find('p').get('class')[1]]

                Cursor=connection.cursor()
                Cursor.execute('insert into books  (Book_name,Price,Rate,category_name)values (?,?,?,?)',[Book_name,Price,Rate,category_name])
                connection.commit()
connection.close() 

        

In [16]:
connection=sqlite3.connect('books.db')
df=pd.read_sql('select * from books',connection)
connection.close()
df


,Book_name,Price,Rate,category_name
0,It's Only the Himalayas,45.17,2,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel
2,See America: A Celebration of Our National Par...,48.87,3,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel
4,Under the Tuscan Sun,37.33,3,Travel
...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,52.65,4,Politics
996,Equal Is Unfair: America's Misguided Fight Aga...,56.86,1,Politics
997,Amid the Chaos,36.58,1,Cultural
998,Dark Notes,19.19,5,Erotica
